# ZipCode Model Creation

## Import libraries and datasets

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_squared_error
from math import sqrt
from math import log
from math import exp
from scipy.stats import boxcox 
from pandas import DataFrame
from pandas import Grouper
from pandas import Series
from pandas import concat
from pandas.plotting import lag_plot
from matplotlib import pyplot
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.arima_model import ARIMAResults
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.graphics.gofplots import qqplot
import warnings

In [2]:
df = pd.read_csv('data/zillow_data.csv')
df_income = pd.read_csv('data/zip_codes_2019_median_inc.csv')

In [3]:
df.columns = df.columns.str.strip().str.lower()
df_income.columns = df_income.columns.str.strip().str.lower()

## Standardizing data/fixing col names

In [4]:
df.rename(columns={'regionname': 'zipcode', 'countyname':'county'},inplace=True)
df_income.rename(columns={
    'zip code':'zipcode', 'population (2019)': 'population',
    'median family income (2019)': 'median_family_income'},
                 inplace=True)

In [5]:
df = df[df['state']=='TX']
df_income['state'] = df_income['preferred name'].str.slice(-2)
df_income = df_income[df_income['state']=='TX']
df_income = (df_income[df_income['type'] != 'PO box'])

In [6]:
df_income.head()

,zipcode,type,state fips,preferred name,alternate names,population,housing units (2019),median_family_income,mfi percentile (2019),latitude,longitude,land area,water area,state
31142,73301,unique,48,"Austin, TX",Irs Service Center,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TX
31143,73344,unique,48,"Austin, TX",Irs Service Center,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TX
31744,75001,standard,48,"Addison, TX",NaN,"14,992","9,298","$79,551",68.0,32.959999,-96.838997,3.834,0.003,TX
31745,75002,standard,48,"Allen, TX","Lucas, Parker","71,253","23,314","$105,656",88.0,33.090000,-96.609001,37.179,2.056,TX
31746,75006,standard,48,"Carrollton, TX",NaN,"51,642","19,178","$72,325",57.0,32.962002,-96.899002,16.875,0.254,TX


## Adding some statistical columns

In [7]:
#Calculate historical return on investment
df['ROI']= (df['2018-04']/df['1996-04'])-1
df['ROI']


#Calculate standard deviation of monthly values
df['std']=df.loc[:,'1996-04':'2018-04'].std(skipna=True, axis=1)

#Calculate historical mean value
df['mean']=df.loc[:,'1996-04':'2018-04'].mean(skipna=True, axis=1)

#Calculate coefficient of variance
df['CV']=df['std']/df['mean']

#Show calculated values
df[['zipcode','std','mean','ROI','CV']].head()

#Descriptive statistics of coefficients of variance.
print(df.CV.describe())

#Define upper limit of CV according to risk profile.
upper_cv = df.CV.quantile(.6)
print(f'\nCV upper limit: {upper_cv}')

#Get the 5 zipcodes with highest ROIs within the firms risk profile.
zc_best5 = df[df['CV']<upper_cv].sort_values('ROI',axis=0,ascending=False)[:5]
print('\n Best 5 Zipcodes:')
zc_best5[['zipcode','ROI','CV']]

warnings.filterwarnings

count    989.000000
mean       0.167433
std        0.041415
min        0.045182
25%        0.142279
50%        0.165783
75%        0.189448
max        0.361446
Name: CV, dtype: float64

CV upper limit: 0.17479440936725246

 Best 5 Zipcodes:


<function warnings.filterwarnings(action, message='', category=<class 'Warning'>, module='', lineno=0, append=False)>

## More cleaning, renaming, filtering

In [8]:
# RegionID is something from Zillow, don't need it and we don't need state any more.
df.drop(['regionid', 'sizerank', 'metro'], axis=1,inplace=True)

In [9]:
# we only want zipcode, pop, mfi, lat and long from 2019 income data
df_income = df_income[df_income.columns[np.r_[0,5,7,9:11]]]
df_income

,zipcode,population,median_family_income,latitude,longitude
31142,73301,NaN,NaN,NaN,NaN
31143,73344,NaN,NaN,NaN,NaN
31744,75001,"14,992","$79,551",32.959999,-96.838997
31745,75002,"71,253","$105,656",33.090000,-96.609001
31746,75006,"51,642","$72,325",32.962002,-96.899002
...,...,...,...,...,...
34257,79961,NaN,NaN,NaN,NaN
34258,79968,NaN,NaN,NaN,NaN
34259,79976,NaN,NaN,NaN,NaN
34260,79978,NaN,NaN,NaN,NaN


In [10]:
# fixing mfi and pop columns
df_income['median_family_income'] = df_income['median_family_income'].str.strip('$\n\t')
df_income['median_family_income'] = df_income['median_family_income'].str.replace(',','')
df_income = df_income[df_income['median_family_income'].notna()]
df_income['median_family_income'] = df_income['median_family_income'].astype(int)
df_income['population'] = df_income['population'].str.replace(',','')
df_income['population'] = df_income['population'].astype(int)

<ipython-input-10-07f5f36c8a35>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_income['median_family_income'] = df_income['median_family_income'].astype(int)
<ipython-input-10-07f5f36c8a35>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_income['population'] = df_income['population'].str.replace(',','')
<ipython-input-10-07f5f36c8a35>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

In [11]:
df_income.head()

,zipcode,population,median_family_income,latitude,longitude
31744,75001,14992,79551,32.959999,-96.838997
31745,75002,71253,105656,33.090000,-96.609001
31746,75006,51642,72325,32.962002,-96.899002
31747,75007,55500,96094,33.005001,-96.897003
31748,75009,14089,117857,33.339001,-96.752998


In [12]:
def melt_data(df):
    melted = pd.melt(df, id_vars=['zipcode', 'city', 'state', 'county','ROI','std','mean','CV'], var_name='date')
    melted['date'] = pd.to_datetime(melted['date'], infer_datetime_format=True)
    melted['year'] = [d.year for d in melted.date]
    melted = melted.dropna(subset=['value'])
    return melted

In [13]:
df = melt_data(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 255096 entries, 0 to 262084
Data columns (total 11 columns):
 #   Column   Non-Null Count   Dtype         
---  ------   --------------   -----         
 0   zipcode  255096 non-null  int64         
 1   city     255096 non-null  object        
 2   state    255096 non-null  object        
 3   county   255096 non-null  object        
 4   ROI      250160 non-null  float64       
 5   std      255096 non-null  float64       
 6   mean     255096 non-null  float64       
 7   CV       255096 non-null  float64       
 8   date     255096 non-null  datetime64[ns]
 9   value    255096 non-null  float64       
 10  year     255096 non-null  int64         
dtypes: datetime64[ns](1), float64(5), int64(2), object(3)
memory usage: 23.4+ MB


In [14]:
df.set_index(df['date'], inplace = True)
df.drop('date',axis=1, inplace=True)
df

,zipcode,city,state,county,ROI,std,mean,CV,value,year
date,,,,,,,,,,
1996-04-01,75070,McKinney,TX,Collin,0.365295,33537.101427,219655.849057,0.152680,235700.0,1996
1996-04-01,77494,Katy,TX,Harris,0.567966,37730.794353,262110.566038,0.143950,210400.0,1996
1996-04-01,79936,El Paso,TX,El Paso,0.571798,18167.079218,101875.471698,0.178326,77300.0,1996
1996-04-01,77084,Houston,TX,Harris,0.729474,17242.109812,121185.283019,0.142279,95000.0,1996
1996-04-01,77449,Katy,TX,Harris,0.846960,18603.288493,126530.188679,0.147026,95400.0,1996
...,...,...,...,...,...,...,...,...,...,...
2018-04-01,76941,Mertzon,TX,Irion,NaN,13734.029878,99318.367347,0.138283,121600.0,2018
2018-04-01,79313,Anton,TX,Hockley,NaN,5286.674724,54424.675325,0.097137,63300.0,2018
2018-04-01,79355,Plains,TX,Yoakum,NaN,12929.465872,78967.346939,0.163732,93500.0,2018


## Merging dfs on zip

In [15]:
df_merged = df.reset_index().merge(df_income, how='left').set_index('date')

In [16]:
df_merged.describe().T

,count,mean,std,min,25%,50%,75%,max
zipcode,255096.0,77163.777546,1361.342308,75001.000000,76017.000000,77098.000000,78251.000000,7.993800e+04
ROI,250160.0,0.940549,0.418125,-0.375469,0.721048,0.903395,1.101759,6.550360e+00
std,255096.0,23762.532113,16134.709292,2684.773009,13998.832825,19663.932774,29134.083926,1.437548e+05
mean,255096.0,140540.600793,94822.092356,32718.292683,81482.641509,115224.150943,165333.207547,8.802596e+05
CV,255096.0,0.168955,0.040480,0.045182,0.144163,0.166917,0.189818,3.614458e-01
value,255096.0,140540.600793,99061.016727,24100.000000,79900.000000,113700.000000,165100.000000,1.268600e+06
year,255096.0,2006.908003,6.393508,1996.000000,2001.000000,2007.000000,2012.000000,2.018000e+03
population,252181.0,25939.370892,20385.536014,1056.000000,9761.000000,21879.000000,37281.000000,1.282940e+05
median_family_income,252181.0,77689.448301,32642.262099,21169.000000,56108.000000,69722.000000,90867.000000,2.500010e+05
latitude,252181.0,31.108154,1.797547,25.944000,29.736000,30.805000,32.723999,3.604600e+01


## Filtering merged df

In [17]:
# Select the first quantile
q1 = df_merged['value'].quantile(.25)

# Select the third quantile
q3 = df_merged['value'].quantile(.75)


# Create a filtered in between q1 & q3
filtered = df_merged[(df_merged['value']>=q1) 
             & (df_merged['value']<=q3)]
df_merged = pd.DataFrame(filtered)
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 127696 entries, 1996-04-01 to 2018-04-01
Data columns (total 14 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   zipcode               127696 non-null  int64  
 1   city                  127696 non-null  object 
 2   state                 127696 non-null  object 
 3   county                127696 non-null  object 
 4   ROI                   125745 non-null  float64
 5   std                   127696 non-null  float64
 6   mean                  127696 non-null  float64
 7   CV                    127696 non-null  float64
 8   value                 127696 non-null  float64
 9   year                  127696 non-null  int64  
 10  population            126507 non-null  float64
 11  median_family_income  126507 non-null  float64
 12  latitude              126507 non-null  float64
 13  longitude             126507 non-null  float64
dtypes: float64(9), int64(2), object(3)
m

## Creating an Affordability Score

In [18]:
# group by date and aggregate on median value
df_merged['median_house_price'] = df_merged.groupby('date').aggregate({'value':'median'})

In [19]:
df_merged['affordability'] = df_merged['value'] -df_merged['median_family_income']/.12

In [20]:
# create function to normalize data
def normalize_data(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))
df_merged['affordability_score']  = normalize_data(df_merged['affordability'])
df_merged

,zipcode,city,state,county,ROI,std,mean,CV,value,year,population,median_family_income,latitude,longitude,median_house_price,affordability,affordability_score
date,,,,,,,,,,,,,,,,,
1996-04-01,77084,Houston,TX,Harris,0.729474,17242.109812,121185.283019,0.142279,95000.0,1996,107673.0,70460.0,29.827000,-95.660004,106600.0,-492166.666667,0.591511
1996-04-01,77449,Katy,TX,Harris,0.846960,18603.288493,126530.188679,0.147026,95400.0,1996,128294.0,82716.0,29.837999,-95.734001,106600.0,-593900.000000,0.484277
1996-04-01,78660,Pflugerville,TX,Travis,0.748020,24894.592870,168193.584906,0.148012,138900.0,1996,91300.0,99733.0,30.440001,-97.595001,106600.0,-692208.333333,0.380652
1996-04-01,77573,League City,TX,Galveston,0.848656,29647.359699,183261.509434,0.161776,141400.0,1996,88131.0,121943.0,29.504000,-95.086998,106600.0,-874791.666667,0.188196
1996-04-01,77584,Pearland,TX,Brazoria,0.851986,27064.756641,188628.301887,0.143482,138500.0,1996,90372.0,126143.0,29.545000,-95.350998,106600.0,-912691.666667,0.148247
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-04-01,79019,Claude,TX,Armstrong,NaN,6615.560732,76994.897959,0.085922,84400.0,2018,1871.0,78438.0,35.007000,-101.393997,123500.0,-569250.000000,0.510260
2018-04-01,77577,Liverpool,TX,Brazoria,1.457990,22188.730901,96658.867925,0.229557,149200.0,2018,1481.0,76382.0,29.284000,-95.281998,123500.0,-487316.666667,0.596623
2018-04-01,77663,Kountze,TX,Hardin,0.727907,17632.899715,120896.226415,0.145852,148600.0,2018,NaN,NaN,NaN,NaN,123500.0,NaN,NaN


In [21]:
df_merged.describe()

,zipcode,ROI,std,mean,CV,value,year,population,median_family_income,latitude,longitude,median_house_price,affordability,affordability_score
count,127696.000000,125745.000000,127696.000000,127696.000000,127696.000000,127696.000000,127696.000000,126507.000000,126507.000000,126507.000000,126507.000000,127696.000000,1.265070e+05,126507.000000
mean,77100.488324,0.989690,20361.176482,117458.198761,0.173059,115703.177077,2007.094647,27131.194361,70592.091418,31.113763,-97.117212,113478.569415,-4.726570e+05,0.612076
std,1326.096965,0.329758,7029.732711,29519.775502,0.036880,24093.633061,6.107367,20426.545392,17395.133355,1.747035,2.282024,3634.448439,1.336489e+05,0.140876
min,75002.000000,-0.121860,4535.065102,54346.415094,0.052476,79900.000000,1996.000000,1056.000000,31396.000000,25.962999,-106.608002,106100.000000,-1.053333e+06,0.000000
25%,76017.000000,0.797568,15778.672504,93129.433962,0.149056,93600.000000,2002.000000,10537.000000,58727.000000,29.730000,-97.707001,111800.000000,-5.527500e+05,0.527652
50%,77086.000000,0.956236,19484.063014,115224.150943,0.170967,113700.000000,2007.000000,23834.000000,69063.000000,31.017000,-96.931999,113600.000000,-4.636167e+05,0.621605
75%,78155.000000,1.127396,23421.858587,137466.037736,0.190941,135500.000000,2012.000000,39183.000000,81065.000000,32.666000,-95.489998,115600.000000,-3.798917e+05,0.709857
max,79938.000000,6.550360,82314.082050,279264.528302,0.361446,165100.000000,2018.000000,128294.000000,143656.000000,36.046001,-93.615997,123800.000000,-1.046333e+05,1.000000


In [22]:
# filtering to the 20th percentile
top_20 = df_merged['affordability_score'].quantile(.2)
top_20

0.5027230244896526

In [23]:
df_merged_20 = df_merged[(df_merged['affordability_score'] < top_20)] 
df_merged_20

,zipcode,city,state,county,ROI,std,mean,CV,value,year,population,median_family_income,latitude,longitude,median_house_price,affordability,affordability_score
date,,,,,,,,,,,,,,,,,
1996-04-01,77449,Katy,TX,Harris,0.846960,18603.288493,126530.188679,0.147026,95400.0,1996,128294.0,82716.0,29.837999,-95.734001,106600.0,-593900.000000,0.484277
1996-04-01,78660,Pflugerville,TX,Travis,0.748020,24894.592870,168193.584906,0.148012,138900.0,1996,91300.0,99733.0,30.440001,-97.595001,106600.0,-692208.333333,0.380652
1996-04-01,77573,League City,TX,Galveston,0.848656,29647.359699,183261.509434,0.161776,141400.0,1996,88131.0,121943.0,29.504000,-95.086998,106600.0,-874791.666667,0.188196
1996-04-01,77584,Pearland,TX,Brazoria,0.851986,27064.756641,188628.301887,0.143482,138500.0,1996,90372.0,126143.0,29.545000,-95.350998,106600.0,-912691.666667,0.148247
1996-04-01,75052,Grand Prairie,TX,Dallas,0.995040,19370.586349,134230.188679,0.144309,100800.0,1996,95495.0,85220.0,32.665001,-97.025002,106600.0,-609366.666667,0.467974
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-04-01,77665,Winnie,TX,Chambers,1.573333,22133.563016,96645.660377,0.229018,154400.0,2018,6152.0,88917.0,29.813000,-94.416000,123500.0,-586575.000000,0.491998
2018-04-01,77534,Danbury,TX,Brazoria,1.212081,22688.458345,114705.660377,0.197797,164800.0,2018,2256.0,107262.0,29.232000,-95.314003,123500.0,-729050.000000,0.341819
2018-04-01,79068,Panhandle,TX,Carson,0.878261,12159.925347,80253.584906,0.151519,108000.0,2018,3489.0,99922.0,35.347000,-101.444000,123500.0,-724683.333333,0.346421


In [24]:
df_merged_20.reset_index().isna().sum()

date                     0
zipcode                  0
city                     0
state                    0
county                   0
ROI                     29
std                      0
mean                     0
CV                       0
value                    0
year                     0
population               0
median_family_income     0
latitude                 0
longitude                0
median_house_price       0
affordability            0
affordability_score      0
dtype: int64

In [25]:
df_merged_20 = df_merged['2017-01-01':].dropna()

In [26]:
#Descriptive statistics of coefficients of variance.
print(df_merged_20.CV.describe())

#Define upper limit of CV according to risk profile.
upper_cv = df_merged_20.CV.quantile(.6)
print(f'\nCV upper limit: {upper_cv}')

#Get the 5 zipcodes with highest ROIs within the firms risk profile.
df_20_cv = df_merged_20[df_merged_20['CV']<upper_cv] 

df_20_cv = df_20_cv.reset_index()
df_20_cv = df_20_cv.loc[df_20_cv.groupby("zipcode")["ROI"].idxmax()].sort_values('ROI',axis=0,ascending=False)[:5]
df_20_cv.set_index('date',inplace=True)
df_20_cv

count    6653.000000
mean        0.171711
std         0.037776
min         0.084827
25%         0.148609
50%         0.172273
75%         0.190743
max         0.315077
Name: CV, dtype: float64

CV upper limit: 0.18002144081780114


,zipcode,city,state,county,ROI,std,mean,CV,value,year,population,median_family_income,latitude,longitude,median_house_price,affordability,affordability_score
date,,,,,,,,,,,,,,,,,
2017-01-01,77020,Houston,TX,Harris,1.641221,11250.658795,65907.924528,0.170703,83200.0,2017,26357.0,35034.0,29.773001,-95.314003,120400.0,-208750.000000,0.890253
2018-03-01,76104,Fort Worth,TX,Tarrant,1.304225,9298.895970,54346.415094,0.171104,80200.0,2018,18344.0,36300.0,32.729000,-97.318001,123300.0,-222300.000000,0.875971
2017-01-01,75062,Irving,TX,Dallas,1.261283,20861.130887,117610.566038,0.177375,155800.0,2017,49306.0,61189.0,32.847000,-96.958000,120400.0,-354108.333333,0.737035
2017-01-01,75050,Grand Prairie,TX,Dallas,1.260028,17806.245475,99366.792453,0.179197,138200.0,2017,43174.0,61583.0,32.772999,-97.004997,120400.0,-374991.666667,0.715022
2017-01-01,75956,Kirbyville,TX,Jasper,1.255220,11219.508619,65782.264151,0.170555,89400.0,2017,8937.0,57196.0,30.676001,-93.983002,120400.0,-387233.333333,0.702119
